In [5]:
import heapq
from functools import cache

from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter, deque
import re
from z3 import Ints, Solver, sat
from tqdm import tqdm

year = 2024
day = 21

download_input(year, day)

In [6]:
aoc, lines, G, DIR, C = read_input(day, test=False)

['985A', '540A', '463A', '671A', '382A']
5 4


In [337]:
from functools import lru_cache
import itertools


@lru_cache(None)
def calc_moves(start, end, keypad_type):

    @lru_cache(None)
    def explore_paths(p, n):
        if p == n:
            return [()]  # Base case: found the target, return an empty path to build on

        paths = []
        start_dist = abs(p[0] - n[0]) + abs(p[1] - n[1])

        for key, d in dir_dict.items():
            pn = (p[0] + d[0], p[1] + d[1])
            dist = abs(pn[0] - n[0]) + abs(pn[1] - n[1])
            if is_valid(pn, keypad_type) and dist <= start_dist:
                for sub_path in explore_paths(pn, n):
                    paths.append((key,) + sub_path)

        return paths

    def is_valid(p, keypad_type):
        if keypad_type == "NUM":
            avoid = (3, 0)
            min_r, max_r = 0, 3
            min_c, max_c = 0, 2
        elif keypad_type == "DIR":
            avoid = (0, 0)
            min_r, max_r = 0, 1
            min_c, max_c = 0, 2
        else:
            return False

        # Check boundaries and avoid certain points
        return min_r <= p[0] <= max_r and min_c <= p[1] <= max_c and p != avoid

    return explore_paths(start, end)


def final_robot(code):
    G = [["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], ["-", "0", "A"]]
    steps = []
    start = (3, 2)
    p = start
    for goal in code:
        if goal != "A":
            row = 2 - ((int(goal) - 1) // 3)
            col = G[row].index(goal)
            n = (row, col)
        else:
            n = (start[0], start[1])
        moves = calc_moves(p, n, "NUM")
        moves = [m + ("A",) for m in moves]
        p = n
        steps.append(moves)

    return steps


dpad_dict = {"A": (0, 2), "v": (1, 1), "^": (0, 1), "<": (1, 0), ">": (1, 2)}


def second_dir_robot(code, start=(0, 2)):

    G = [["-", "^", "A"], ["<", "v", ">"]]
    steps = []
    p = start

    for goal in code:
        moves = steps_to_goal(goal, p)
        steps.append(moves)
        p = dpad_dict[goal]

    return steps


DP = {}


def steps_to_goal(goal, p):
    if (goal, p) in DP:
        return DP[(goal, p)]

    moves = calc_moves(p, dpad_dict[goal], "DIR")
    moves = tuple(m + ("A",) for m in moves)

    DP[(goal, p)] = deepcopy(moves)

    return moves


import time


# @lru_cache(maxsize=None)
def perform_step_from_pos(start, combos1, depth):

    if depth == 0:
        return combos1

    ans = []

    for combination in combos1:
        flat = list(itertools.chain(*combination))
        sol2 = second_dir_robot(flat, start)
        combos2 = (
            tuple(itertools.chain(*combination2))
            for combination2 in itertools.product(*sol2)
        )
        inp = tuple(x for x in combos2)
        res = perform_step_from_pos((0, 2), inp, depth - 1)
        ans.extend(res)

    return ans


all_spots = {"A": (0, 2), "v": (1, 1), "^": (0, 1), "<": (1, 0), ">": (1, 2)}

costs = {}

for spot, start in all_spots.items():

    for target in all_spots.keys():
        solutions = perform_step_from_pos(start, (target,), 2)
        solutions.sort(key=len)
        shortest = solutions[0]
        costs[(spot, target)] = shortest

results = {}

for code in lines:

    solutions = []

    sol1 = final_robot(code)
    combos1 = list(itertools.product(*sol1))
    res = []
    for combination in combos1:
        ans = 0
        flat = list(itertools.chain(*combination))
        for l, r in zip(["A"] + flat, flat):
            # print((l,r), costs[(l,r)])
            ans += len(costs[(l, r)])
        # print(f"{flat=} {ans=}")
        res.append(ans)

    results[code] = min(res)


scores = {}
print(results)

for code in lines:
    if code in results:
        code_as_int = int(code.replace("A", ""))

        this_score = code_as_int * results[code]
        scores[code] = this_score

print(sum(scores.values()))

{'985A': 66, '540A': 72, '463A': 70, '671A': 74, '382A': 68}
211930


In [339]:
from functools import cache


# HEAVILY adapted from https://github.com/LiquidFun/adventofcode/blob/main/2024/21/21.py


NUM_ARR = [["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], ["-", "0", "A"]]
DIR_ARR = [["-", "^", "A"], ["<", "v", ">"]]

NUM = {}
for r, row in enumerate(NUM_ARR):
    for c, col in enumerate(row):
        NUM[col] = (r, c)

DIR = {}
for r, row in enumerate(DIR_ARR):
    for c, col in enumerate(row):
        DIR[col] = (r, c)

G_TEST = {}


@cache
def path(p, n):
    pad = NUM if (p in NUM and n in NUM) else DIR
    # avoid = (0,0) if pad == NUM else (3,0)

    start_row, start_col = pad[p]
    end_row, end_col = pad[n]

    dr = end_row - start_row
    dc = end_col - start_col
    row_change = ("^" * -dr) + ("v" * dr)
    col_change = ("<" * -dc) + (">" * dc)

    bad_row, bad_col = pad["-"]
    bad_diff_row, bad_diff_col = bad_row - start_row, bad_col - start_col
    bad = (bad_diff_row, bad_diff_col)

    # prefer vertical movement first if (A or B) and C
    # where A is that you're moving to the right.
    #       B is that you're moving to the bad column from the bad row.
    #       C is that you're not moving to the bad row from the bad column.

    A = dc > 0
    B = bad_diff_col == dc and bad_diff_row == 0
    C = bad != (dr, 0)

    prefer_yy_first = (A or B) and C

    return (
        row_change + col_change if prefer_yy_first else col_change + row_change
    ) + "A"


@cache
def length(code, depth, s=0):
    if depth == 0:
        return len(code)
    for i, c in enumerate(code):
        s += length(path(code[i - 1], c), depth - 1)
    return s


print(sum(int(code[:-1]) * length(code, 3) for code in lines))
print(sum(int(code[:-1]) * length(code, 26) for code in lines))

211930
263492840501566
